In [1]:
# Keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Embedding, GRU

# Other
import re
import string
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [26]:
def saveModel(model):
    # serialize model to YAML
    model_yaml = model.to_yaml()
    with open("Summary.yaml", "w") as yaml_file:
        yaml_file.write(model_yaml)
    # serialize weights to HDF5
    model.save_weights("Weights.h5")
    print("Saved model to disk")

In [2]:
vocabulary_size = 10000

In [3]:
path = "../Clean/lemma_allresult.csv"
dataset = pd.read_csv(path)

dataset.shape

(10000, 2)

In [4]:
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(dataset['cleaned_review'])

sequences = tokenizer.texts_to_sequences(dataset['cleaned_review'])
feat = pad_sequences(sequences, maxlen=853)
target = dataset['Label'].values

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(feat,target, test_size = 0.2, train_size = 0.8, random_state = 42)

In [8]:
Y_train.shape

(8000,)

In [9]:
embeddings_index = dict()
f = open('glove.6B/glove.6B.100d.txt',encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [10]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [11]:
model_glove = Sequential()
model_glove.add(Embedding(input_dim=vocabulary_size,output_dim=100,input_length=853, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(GRU(100))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
model_glove.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 853, 100)          1000000   
_________________________________________________________________
dropout (Dropout)            (None, 853, 100)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 849, 64)           32064     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 212, 64)           0         
_________________________________________________________________
gru (GRU)                    (None, 100)               49800     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 1,081,965
Trainable params: 81,965
Non-trainable params: 1,000,000
_________________________________________

In [13]:
model_glove.fit(X_train,
                Y_train,
                validation_split=0.4, 
                epochs = 10,
                verbose=1,
                batch_size=32)

Train on 4800 samples, validate on 3200 samples
Epoch 1/10
4800/4800 [==============================] - 32s 7ms/sample - loss: 0.6048 - accuracy: 0.6571 - val_loss: 0.4605 - val_accuracy: 0.7884
Epoch 2/10
4800/4800 [==============================] - 31s 7ms/sample - loss: 0.4741 - accuracy: 0.7748 - val_loss: 0.4163 - val_accuracy: 0.8087
Epoch 3/10
4800/4800 [==============================] - 30s 6ms/sample - loss: 0.4354 - accuracy: 0.7942 - val_loss: 0.4043 - val_accuracy: 0.8163
Epoch 4/10
4800/4800 [==============================] - 30s 6ms/sample - loss: 0.3616 - accuracy: 0.8373 - val_loss: 0.4207 - val_accuracy: 0.8081
Epoch 5/10
4800/4800 [==============================] - 30s 6ms/sample - loss: 0.3049 - accuracy: 0.8675 - val_loss: 0.4092 - val_accuracy: 0.8256
Epoch 6/10
4800/4800 [==============================] - 31s 6ms/sample - loss: 0.2596 - accuracy: 0.8888 - val_loss: 0.4091 - val_accuracy: 0.8263
Epoch 7/10
4800/4800 [==============================] - 31s 6ms/sample

In [14]:
score,acc = model_glove.evaluate(X_test, Y_test, verbose = 1, batch_size = 32)

2000/2000 [==============================] - 4s 2ms/sample - loss: 0.6288 - accuracy: 0.7965


In [15]:
test_path = "../Clean/lemma10000_master_result.csv"

test_dataset = pd.read_csv(test_path)

In [16]:
#tokenizer.fit_on_texts(test_dataset['cleaned_review'])

test_sequences = tokenizer.texts_to_sequences(test_dataset['cleaned_review'])
test_feat = pad_sequences(test_sequences, maxlen=853)
test_target = test_dataset['Label'].values

In [17]:
result = model_glove.predict(test_feat)

In [18]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true=test_target , y_pred=result>0.5)

cm

array([[3653, 1347],
       [ 478, 4522]], dtype=int64)

In [27]:
saveModel(model_glove)

Saved model to disk
